In [1]:
import numpy as np
from joblib import Parallel, delayed
from itertools import product

import sys
sys.path.append('/home/yuncong/Brain/pipeline_scripts')
from utilities2014 import *

dm = DataManager(generate_hierarchy=False, stack='RS141', resol='x5', section=17)
dm._load_image()
textonmap = dm.load_pipeline_result('texMap', 'npy')
n_texton = len(np.unique(textonmap)) - 1

image = dm.image_rgb
mask = dm.mask

height, width = image.shape[:2]

n_theta = 12
theta_binwidth = 2*np.pi/n_theta

radius = 100

thetas = np.linspace(2*np.pi, n_theta)

box_indices = np.dstack(np.meshgrid(np.arange(-radius, radius+1), np.arange(-radius, radius+1))).astype(np.int16)
circle_indices = box_indices[np.linalg.norm(box_indices, axis=-1) <= radius]
angles = np.arctan2(circle_indices[:,0], circle_indices[:,1])
angles[angles < 0] += 2*np.pi
angular_bins = np.int0(angles/theta_binwidth)
pie_indices = [circle_indices[angular_bins==k] for k in range(n_theta)]

import compute_pie

mys, mxs = np.where(mask)
mys = mys.astype(np.int16)
mxs = mxs.astype(np.int16)

n_orientation = int(n_theta/2)

H = np.zeros((n_orientation, height, width, n_texton), np.int)

for s in range(n_theta):
    qq = time.time()
    Hs = compute_pie.compute_pie_histogram(textonmap,
                          pie_indices[s], 
                          mys, 
                          mxs,
                          radius, 
                          height, 
                          width,
                          n_texton)
    H[s] = Hs
    print time.time() - qq

0
500000
1000000
1500000
2000000
2500000
3000000
3500000
4000000
4500000
5000000
5500000
6000000
6500000
7000000
7500000
8000000
8500000
9000000
9500000
205.678580999
0
500000
1000000
1500000
2000000
2500000
3000000
3500000
4000000
4500000
5000000
5500000
6000000
6500000
7000000
7500000
8000000
8500000
9000000
9500000
177.280751944
0
500000
1000000
1500000
2000000
2500000
3000000
3500000
4000000
4500000
5000000
5500000
6000000
6500000
7000000
7500000
8000000
8500000
9000000
9500000
197.626872063
0
500000
1000000
1500000
2000000
2500000
3000000
3500000
4000000
4500000
5000000
5500000


KeyboardInterrupt: 

In [9]:
from collections import Counter

def compute_pie_histogram(textonmap, pie_indices_s, mys, mxs,
                          radius, 
                          height, 
                          width):
    Hs = np.zeros((height, width, n_texton), np.int16)

    for yc,xc in zip(mys, mxs):
#         qq = time.time()
        ys = pie_indices_s[:,0] + yc
        xs = pie_indices_s[:,1] + xc
        if yc < radius or yc > height - radius or xc < radius or xc > width - radius:
            sel = (ys>=0)&(ys<height)&(xs>=0)&(xs<width)
            pie_indices_s = pie_indices_s[sel]
            xs = xs[sel]
            ys = ys[sel]
        
#         w = Counter(textonmap[ys, xs])
#         Hs[yc, xc] = [w[i] for i in range(n_texton)]

        Hs[yc, xc], _ = np.histogram(textonmap[ys, xs], bins=range(n_texton+1))
#         print time.time() - qq
    return Hs

mys, mxs = np.where(mask)
mys = mys.astype(np.int16)
mxs = mxs.astype(np.int16)

for s in range(n_theta/2):
    qq = time.time()
    H = compute_pie_histogram(textonmap,
                          pie_indices[s], 
                          mys, 
                          mxs,
                          radius, 
                          height, 
                          width)
    print time.time() - qq